<a href="https://colab.research.google.com/github/arvindsuresh-math/Fall-2025-Team-Big-Data/blob/main/02_modeling_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Airbnb Price Prediction: Full Prototyping Pipeline for Google Colab**

This notebook contains the end-to-end code for data processing, model building, and training, optimized for a Google Colab GPU environment.

In [3]:
from google.colab import userdata
from huggingface_hub import login
import os

try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Hugging Face login successful.")
except userdata.SecretNotFoundError:
    print("Secret 'HF_TOKEN' not found.")
    print("Please add your Hugging Face token to Colab Secrets.")
except Exception as e:
    print(f"An error occurred during Hugging Face login: {e}")

Hugging Face login successful.


In [4]:
!pip install pandas
!pip install pyarrow
!pip install sentence-transformers
!pip install scikit-learn
!pip install torch
!pip install tqdm

#### **2. Configuration and Data Upload**

Run this cell to set up the main configuration. **After running it, upload your `.parquet` file** using the "Files" tab on the left-hand side of the Colab interface.

In [5]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import json

# --- Main Configuration ---
CITY = "nyc"
DATASET_FILENAME = f"{CITY}_final_modeling_dataset.parquet"
DATASET_PATH = f"./{DATASET_FILENAME}" # Assumes file is uploaded to the root of the Colab runtime

VAL_SIZE = 0.2
RANDOM_STATE = 42
BATCH_SIZE = 256
LEARNING_RATE = 1e-3
N_EPOCHS = 5

# --- Device Selection for Colab GPU ---
if torch.cuda.is_available():
    DEVICE = "cuda"
    print(f"GPU found: {torch.cuda.get_device_name(0)}. Using CUDA for acceleration.")
else:
    DEVICE = "cpu"
    print("GPU not available. Using CPU.")

# Configure pandas display
pd.options.display.max_columns = 100

GPU found: Tesla T4. Using CUDA for acceleration.


#### **3. Data Loading and Stratified Split**

This step loads the dataset and performs the stratified split to create training and validation sets.

In [6]:
# --- Load Data ---
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"'{DATASET_FILENAME}' not found. Please upload the file.")

print(f"Loading final dataset from: {DATASET_PATH}")
df = pd.read_parquet(DATASET_PATH)
print("Dataset loaded successfully.")

# --- Stratified Train/Validation Split ---
stratify_col = df['neighbourhood_cleansed'].astype(str) + '_' + df['month'].astype(str)
strata_counts = stratify_col.value_counts()
valid_strata = strata_counts[strata_counts >= 2].index
df_filtered = df[stratify_col.isin(valid_strata)].copy()

print(f"\nRemoved {len(df) - len(df_filtered):,} records belonging to strata with only 1 member.")

train_indices, val_indices = train_test_split(
    df_filtered.index,
    test_size=VAL_SIZE,
    random_state=RANDOM_STATE,
    stratify=df_filtered['neighbourhood_cleansed'].astype(str) + '_' + df_filtered['month'].astype(str)
)

train_df = df_filtered.loc[train_indices].copy().reset_index(drop=True)
val_df = df_filtered.loc[val_indices].copy().reset_index(drop=True)

print(f"Training records: {len(train_df):,}, Validation records: {len(val_df):,}")

Loading final dataset from: ./nyc_final_modeling_dataset.parquet
Dataset loaded successfully.

Removed 230 records belonging to strata with only 1 member.
Training records: 66,390, Validation records: 16,598


#### **4. FeatureProcessor Class**

The class for handling all feature transformations.

In [7]:
class FeatureProcessor:
    def __init__(self, embedding_dim_geo: int = 32):
        self.vocabs, self.scalers = {}, {}
        self.embedding_dim_geo = embedding_dim_geo
        self.categorical_cols = ["neighbourhood_cleansed", "property_type", "room_type", "bathrooms_type", "bedrooms", "beds", "bathrooms_numeric"]
        self.numerical_cols = ["accommodates", "review_scores_rating", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "host_response_rate", "host_acceptance_rate"]
        self.log_transform_cols = ["number_of_reviews_ltm"]
        self.boolean_cols = ["host_is_superhost", "host_identity_verified", "instant_bookable"]

    def _create_positional_encoding(self, value, max_val):
        d = self.embedding_dim_geo
        if d % 2 != 0: raise ValueError("embedding_dim_geo must be even.")
        pe = np.zeros(d)
        position = (value / max_val) * 10000
        div_term = np.exp(np.arange(0, d, 2) * -(np.log(10000.0) / d))
        pe[0::2] = np.sin(position * div_term)
        pe[1::2] = np.cos(position * div_term)
        return pe

    def fit(self, df: pd.DataFrame):
        for col in self.categorical_cols:
            valid_uniques = df[col].dropna().unique().tolist()
            self.vocabs[col] = {val: i for i, val in enumerate(["<UNK>"] + sorted(valid_uniques))}
        for col in self.numerical_cols + self.log_transform_cols:
            vals = np.log1p(df[col]) if col in self.log_transform_cols else df[col]
            self.scalers[col] = {'mean': vals.mean(), 'std': vals.std()}

    def transform(self, df: pd.DataFrame) -> dict:
        df = df.copy()
        half_dim = self.embedding_dim_geo // 2
        lat_enc = df['latitude'].apply(lambda x: self._create_positional_encoding(x, 90)[:half_dim])
        lon_enc = df['longitude'].apply(lambda x: self._create_positional_encoding(x, 180)[:half_dim])

        location = {"geo_position": np.hstack([np.stack(lat_enc), np.stack(lon_enc)]), "neighbourhood": df["neighbourhood_cleansed"].apply(lambda x: self.vocabs["neighbourhood_cleansed"].get(x, 0)).values}
        size = {col: df[col].apply(lambda x: self.vocabs[col].get(x, 0) if pd.notna(x) else 0).values for col in ["property_type", "room_type", "bathrooms_type", "bedrooms", "beds", "bathrooms_numeric"]}
        size["accommodates"] = ((df["accommodates"] - self.scalers["accommodates"]["mean"]) / self.scalers["accommodates"]["std"]).values
        quality = {col: ((df[col] - self.scalers[col]["mean"]) / self.scalers[col]["std"]).values for col in self.numerical_cols if col != "accommodates"}
        quality["number_of_reviews_ltm"] = ((np.log1p(df["number_of_reviews_ltm"]) - self.scalers["number_of_reviews_ltm"]["mean"]) / self.scalers["number_of_reviews_ltm"]["std"]).values
        for col in self.boolean_cols: quality[col] = df[col].astype(float).values
        seasonality = {"cyclical": np.vstack([np.sin(2 * np.pi * df["month"] / 12), np.cos(2 * np.pi * df["month"] / 12)]).T}

        return {"location": location, "size_capacity": size, "quality": quality, "amenities": {"text": df["amenities"].tolist()}, "seasonality": seasonality, "target_price": df["target_price"].values, "sample_weight": df["estimated_occupancy_rate"].values}

# Instantiate, fit, and transform
processor = FeatureProcessor()
processor.fit(train_df)
train_features = processor.transform(train_df)
val_features = processor.transform(val_df)
print("\nFeature processing complete.")


Feature processing complete.


#### **5. PyTorch Dataset and DataLoader**

This section defines the data loading and batching pipeline.

In [8]:
class AirbnbPriceDataset(Dataset):
    def __init__(self, features: dict):
        self.features = features
        self.n_samples = len(features['target_price'])

    def __len__(self): return self.n_samples

    def __getitem__(self, index: int) -> dict:
        item = {}
        item['loc_geo_position'] = torch.tensor(self.features['location']['geo_position'][index], dtype=torch.float32)
        item['loc_neighbourhood'] = torch.tensor(self.features['location']['neighbourhood'][index], dtype=torch.long)
        for k, v in self.features['size_capacity'].items(): item[f'size_{k}'] = torch.tensor(v[index], dtype=torch.float32 if k == 'accommodates' else torch.long)
        for k, v in self.features['quality'].items(): item[f'qual_{k}'] = torch.tensor(v[index], dtype=torch.float32)
        item['amenities_text'] = self.features['amenities']['text'][index]
        item['season_cyclical'] = torch.tensor(self.features['seasonality']['cyclical'][index], dtype=torch.float32)
        item['target_price'] = torch.tensor(self.features['target_price'][index], dtype=torch.float32)
        item['sample_weight'] = torch.tensor(self.features['sample_weight'][index], dtype=torch.float32)
        return item

# The tokenizer is instantiated to use the GPU (DEVICE)
tokenizer_model = SentenceTransformer('BAAI/bge-small-en-v1.5', device=DEVICE)

def custom_collate_fn(batch: list) -> dict:
    amenities_texts = [item.pop('amenities_text') for item in batch]
    collated_batch = {key: torch.stack([d[key] for d in batch]) for key in batch[0].keys()}
    tokenized = tokenizer_model.tokenizer(amenities_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
    collated_batch['amenities_tokens'] = tokenized
    return collated_batch

train_dataset = AirbnbPriceDataset(train_features)
val_dataset = AirbnbPriceDataset(val_features)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn)

print(f"DataLoaders created. Batches per epoch: Train={len(train_loader)}, Val={len(val_loader)}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

DataLoaders created. Batches per epoch: Train=260, Val=65


#### **6. Additive Model Architecture**

This is the `nn.Module` class for our 5-axis additive model. It is configured to run on the selected `DEVICE`.

In [13]:
class AdditiveAxisModel(nn.Module):
    def __init__(self, processor: FeatureProcessor, device: str):
        super().__init__()
        self.vocabs, self.device = processor.vocabs, device

        # --- Embeddings ---
        self.embed_neighbourhood = nn.Embedding(len(self.vocabs['neighbourhood_cleansed']), 16)
        self.embed_property_type = nn.Embedding(len(self.vocabs['property_type']), 8)
        self.embed_room_type = nn.Embedding(len(self.vocabs['room_type']), 4)
        self.embed_bathrooms_type = nn.Embedding(len(self.vocabs['bathrooms_type']), 2)
        self.embed_bedrooms = nn.Embedding(len(self.vocabs['bedrooms']), 4)
        self.embed_beds = nn.Embedding(len(self.vocabs['beds']), 4)
        self.embed_bathrooms_numeric = nn.Embedding(len(self.vocabs['bathrooms_numeric']), 4)

        # --- Amenities Transformer (placed on the correct device) ---
        self.amenities_transformer = SentenceTransformer('BAAI/bge-small-en-v1.5', device=self.device)
        for param in self.amenities_transformer.parameters(): param.requires_grad = False

        # --- Sub-Networks ---
        self.loc_subnet = nn.Sequential(nn.Linear(48, 32), nn.ReLU(), nn.Linear(32, 1))
        self.size_subnet = nn.Sequential(nn.Linear(27, 32), nn.ReLU(), nn.Linear(32, 1))
        self.qual_subnet = nn.Sequential(nn.Linear(12, 32), nn.ReLU(), nn.Linear(32, 1))
        self.amenities_subnet = nn.Linear(384, 1)
        self.season_subnet = nn.Sequential(nn.Linear(2, 16), nn.ReLU(), nn.Linear(16, 1))
        self.global_bias = nn.Parameter(torch.randn(1))
        self.to(self.device)

    def forward(self, batch: dict) -> torch.Tensor:
        # --- FIX IS HERE ---
        # The original loop does not modify the batch in place correctly.
        # The most robust way is to handle the device placement explicitly for
        # the amenities tokens just before they are used.

        # 1. Move all standard tensors to the model's device
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch[k] = v.to(self.device)

        # 2. Specifically move the amenities tokens dictionary to the device
        amenities_tokens = batch['amenities_tokens']
        for k, v in amenities_tokens.items():
            amenities_tokens[k] = v.to(self.device)

        # --- Process Axes ---
        loc_input = torch.cat([batch['loc_geo_position'], self.embed_neighbourhood(batch['loc_neighbourhood'])], dim=1)
        size_input = torch.cat([self.embed_property_type(batch['size_property_type']), self.embed_room_type(batch['size_room_type']), self.embed_bathrooms_type(batch['size_bathrooms_type']), self.embed_beds(batch['size_beds']), self.embed_bedrooms(batch['size_bedrooms']), self.embed_bathrooms_numeric(batch['size_bathrooms_numeric']), batch['size_accommodates'].unsqueeze(1)], dim=1)
        qual_input = torch.cat([batch['qual_review_scores_rating'].unsqueeze(1), batch['qual_review_scores_cleanliness'].unsqueeze(1), batch['qual_review_scores_checkin'].unsqueeze(1), batch['qual_review_scores_communication'].unsqueeze(1), batch['qual_review_scores_location'].unsqueeze(1), batch['qual_review_scores_value'].unsqueeze(1), batch['qual_host_response_rate'].unsqueeze(1), batch['qual_host_acceptance_rate'].unsqueeze(1), batch['qual_number_of_reviews_ltm'].unsqueeze(1), batch['qual_host_is_superhost'].unsqueeze(1), batch['qual_host_identity_verified'].unsqueeze(1), batch['qual_instant_bookable'].unsqueeze(1)], dim=1)

        # Now, amenities_tokens and the transformer model are both on the same GPU device
        amenities_embed = self.amenities_transformer(amenities_tokens)['sentence_embedding']

        # --- Get Contributions ---
        p_loc = self.loc_subnet(loc_input)
        p_size = self.size_subnet(size_input)
        p_qual = self.qual_subnet(qual_input)
        p_amenities = self.amenities_subnet(amenities_embed)
        p_season = self.season_subnet(batch['season_cyclical'])

        return (self.global_bias + p_loc + p_size + p_qual + p_amenities + p_season).squeeze(-1)

#### **7. Training Loop**

The final step is to instantiate the model and run the training process.

In [14]:
model = AdditiveAxisModel(processor, device=DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print("\n--- Starting Model Training ---")

for epoch in range(N_EPOCHS):
    model.train()
    train_loss = 0.0
    train_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Train]")

    for batch in train_iterator:
        predictions = model(batch)
        targets = batch['target_price'].to(DEVICE)
        weights = batch['sample_weight'].to(DEVICE)
        loss = (weights * (predictions - targets)**2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_loss = 0.0
    val_iterator = tqdm(val_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Val]")
    with torch.no_grad():
        for batch in val_iterator:
            predictions = model(batch)
            targets = batch['target_price'].to(DEVICE)
            weights = batch['sample_weight'].to(DEVICE)
            loss = (weights * (predictions - targets)**2).mean()
            val_loss += loss.item()
            val_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{N_EPOCHS} -> Train W-MSE: {avg_train_loss:.2f}, Val W-MSE: {avg_val_loss:.2f}")

print("\n--- Training Complete ---")


--- Starting Model Training ---


Epoch 1/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.53it/s, loss=10033.57]


Epoch 1/5 -> Train W-MSE: 159919.99, Val W-MSE: 312283.91


Epoch 2/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, loss=6600.26]


Epoch 2/5 -> Train W-MSE: 150499.25, Val W-MSE: 307818.81


...

Epoch 3/5 -> Train W-MSE: 148390.47, Val W-MSE: 306210.13


Epoch 4/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.53it/s, loss=4836.97]


Epoch 4/5 -> Train W-MSE: 147376.51, Val W-MSE: 305171.51


Epoch 5/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.53it/s, loss=4375.31]

Epoch 5/5 -> Train W-MSE: 146668.50, Val W-MSE: 304344.00

--- Training Complete ---


In [16]:
LEARNING_RATE = 1e-2
model_2 = AdditiveAxisModel(processor, device=DEVICE)
optimizer = optim.AdamW(model_2.parameters(), lr=LEARNING_RATE)

print("\n--- Starting Model Training ---")

for epoch in range(N_EPOCHS):
    model_2.train()
    train_loss = 0.0
    train_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Train]")

    for batch in train_iterator:
        predictions = model_2(batch)
        targets = batch['target_price'].to(DEVICE)
        weights = batch['sample_weight'].to(DEVICE)
        loss = (weights * (predictions - targets)**2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_train_loss = train_loss / len(train_loader)

    model_2.eval()
    val_loss = 0.0
    val_iterator = tqdm(val_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Val]")
    with torch.no_grad():
        for batch in val_iterator:
            predictions = model_2(batch)
            targets = batch['target_price'].to(DEVICE)
            weights = batch['sample_weight'].to(DEVICE)
            loss = (weights * (predictions - targets)**2).mean()
            val_loss += loss.item()
            val_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{N_EPOCHS} -> Train W-MSE: {avg_train_loss:.2f}, Val W-MSE: {avg_val_loss:.2f}")

print("\n--- Training Complete ---")


--- Starting Model Training ---


Epoch 1/5 [Val]: 100%|██████████| 65/65 [00:43<00:00,  1.51it/s, loss=3513.71]


Epoch 1/5 -> Train W-MSE: 149073.49, Val W-MSE: 302355.02


Epoch 2/5 [Val]: 100%|██████████| 65/65 [00:43<00:00,  1.51it/s, loss=5099.27]


Epoch 2/5 -> Train W-MSE: 144413.44, Val W-MSE: 299192.04


...

Epoch 3/5 -> Train W-MSE: 142926.05, Val W-MSE: 293573.97


Epoch 4/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.52it/s, loss=5252.26]


Epoch 4/5 -> Train W-MSE: 140520.60, Val W-MSE: 284370.72


Epoch 5/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.52it/s, loss=7762.66]

Epoch 5/5 -> Train W-MSE: 138064.06, Val W-MSE: 276477.98

--- Training Complete ---


In [17]:
LEARNING_RATE = 1e-1
model_3 = AdditiveAxisModel(processor, device=DEVICE)
optimizer = optim.AdamW(model_3.parameters(), lr=LEARNING_RATE)

print("\n--- Starting Model Training ---")

for epoch in range(N_EPOCHS):
    model_3.train()
    train_loss = 0.0
    train_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Train]")

    for batch in train_iterator:
        predictions = model_3(batch)
        targets = batch['target_price'].to(DEVICE)
        weights = batch['sample_weight'].to(DEVICE)
        loss = (weights * (predictions - targets)**2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_train_loss = train_loss / len(train_loader)

    model_3.eval()
    val_loss = 0.0
    val_iterator = tqdm(val_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Val]")
    with torch.no_grad():
        for batch in val_iterator:
            predictions = model_3(batch)
            targets = batch['target_price'].to(DEVICE)
            weights = batch['sample_weight'].to(DEVICE)
            loss = (weights * (predictions - targets)**2).mean()
            val_loss += loss.item()
            val_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{N_EPOCHS} -> Train W-MSE: {avg_train_loss:.2f}, Val W-MSE: {avg_val_loss:.2f}")

print("\n--- Training Complete ---")


--- Starting Model Training ---


Epoch 1/5 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.51it/s, loss=4248.33]


Epoch 1/5 -> Train W-MSE: 144509.35, Val W-MSE: 284081.39


Epoch 2/5 [Val]: 100%|██████████| 65/65 [00:43<00:00,  1.51it/s, loss=3174.48]


Epoch 2/5 -> Train W-MSE: 138679.32, Val W-MSE: 286510.93


...

Epoch 3/5 -> Train W-MSE: 138890.16, Val W-MSE: 255885.86


Epoch 4/5 [Val]: 100%|██████████| 65/65 [00:43<00:00,  1.51it/s, loss=3372.25]


Epoch 4/5 -> Train W-MSE: 140166.90, Val W-MSE: 274841.36


Epoch 5/5 [Val]: 100%|██████████| 65/65 [00:43<00:00,  1.49it/s, loss=12699.57]

Epoch 5/5 -> Train W-MSE: 136655.15, Val W-MSE: 261030.59

--- Training Complete ---


In [18]:
LEARNING_RATE = 1e-2
N_EPOCHS = 20

model_2 = AdditiveAxisModel(processor, device=DEVICE)
optimizer = optim.AdamW(model_2.parameters(), lr=LEARNING_RATE)

print("\n--- Starting Model Training ---")

for epoch in range(N_EPOCHS):
    model_2.train()
    train_loss = 0.0
    train_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Train]")

    for batch in train_iterator:
        predictions = model_2(batch)
        targets = batch['target_price'].to(DEVICE)
        weights = batch['sample_weight'].to(DEVICE)
        loss = (weights * (predictions - targets)**2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_train_loss = train_loss / len(train_loader)

    model_2.eval()
    val_loss = 0.0
    val_iterator = tqdm(val_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Val]")
    with torch.no_grad():
        for batch in val_iterator:
            predictions = model_2(batch)
            targets = batch['target_price'].to(DEVICE)
            weights = batch['sample_weight'].to(DEVICE)
            loss = (weights * (predictions - targets)**2).mean()
            val_loss += loss.item()
            val_iterator.set_postfix({'loss': f"{loss.item():.2f}"})

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{N_EPOCHS} -> Train W-MSE: {avg_train_loss:.2f}, Val W-MSE: {avg_val_loss:.2f}")

print("\n--- Training Complete ---")


--- Starting Model Training ---


Epoch 1/20 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.52it/s, loss=3285.27]


Epoch 1/20 -> Train W-MSE: 149641.67, Val W-MSE: 303257.87


Epoch 2/20 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.52it/s, loss=3431.67]


Epoch 2/20 -> Train W-MSE: 144850.40, Val W-MSE: 300157.12


...

Epoch 18/20 -> Train W-MSE: 131228.84, Val W-MSE: 257474.39


Epoch 19/20 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, loss=4013.96]


Epoch 19/20 -> Train W-MSE: 130706.96, Val W-MSE: 258057.55


Epoch 20/20 [Val]: 100%|██████████| 65/65 [00:42<00:00,  1.53it/s, loss=4172.12]

Epoch 20/20 -> Train W-MSE: 130788.70, Val W-MSE: 257104.22

--- Training Complete ---
